In [922]:
import numpy as np
import math
from scipy.optimize import minimize

In [923]:
m = 46873
delta = 0.02
t = 64

E_qkd = 10**-6

bexp10k = 4060
bexp10k/10240

0.396484375

In [924]:
def h(p):
    if p == 1: return 0
    if p == 0: return 0
    return -p*math.log2(p)-(1-p)*math.log2(1-p)

In [925]:
def gamma_del_xi (xi):
    global m
    global delta 
    term1 = 1.0/(m*(delta+xi) +1)
    term2 = 1.0/ (m - m*(delta + xi) + 1 )
    return term1 + term2 


In [926]:
def E_pe(nu,xi,beta):
    global m
    
    k = m*beta
    n = m - k
    nu_prime = nu - xi 
    term1 = np.exp(-2.0 * m*k*(xi**2) / (n+1)) 
    term2 = np.exp(-2.0 * gamma_del_xi(xi)*( (n*nu_prime)**2 -1) )
    return  np.sqrt(term1+term2)
    

In [927]:
def E_pa (nu,alpha,beta):
    global m
    global delta
    global t
    r = 1.19 *h(delta)
    #r = bexp10k/10240
    
    l = m*alpha
    n = m - m*beta
    exponent = (-n*(1-h(delta+nu)-r)+t+l)
    return np.sqrt(np.power(2.0,exponent) ) / 2.0 


In [928]:
#Optimization code below 

# x = (alpha, beta, nu, xi)
def objective(x):
    global m
    alpha = x[0]
    obj = -(alpha*m)
    return obj


In [929]:
def ineq1 (x):
    alpha = x[0]
    beta = x[1]
    nu = x[2]
    xi = x[3]
    global E_qkd
    global t
    
    lhs = 2**-t + 2*E_pe(nu,xi,beta) + E_pa(nu,alpha,beta)
    return E_qkd - lhs

def ineq2 (x):
    alpha = x[0]
    beta = x[1]
    nu = x[2]
    xi = x[3]
    return nu - xi


In [930]:
#bounds
bnds = ((0,1.0),(0,0.4),(0,0.5-delta),(0,0.5-delta))

x0 = [1.9e-3,0.2,0.11,0.049]

In [931]:
con1 = {'type':'ineq','fun':ineq1}
con2 = {'type':'ineq','fun':ineq2}
cons = [con1,con2]
opts = {"disp": True, "maxiter": 5000}

In [932]:
sol = minimize(objective, x0, method='SLSQP', bounds = bnds, constraints=cons,options=opts)

#sol = minimize(objective, x0, method='Nelder-Mead', bounds = bnds, constraints=cons,options=opts)


Optimization terminated successfully    (Exit mode 0)
            Current function value: -10114.759310063291
            Iterations: 11
            Function evaluations: 45
            Gradient evaluations: 7


<ipython-input-927-6f305fe33eb0>:11: RuntimeWarning: overflow encountered in power
  return np.sqrt(np.power(2.0,exponent) ) / 2.0


In [933]:
sol

     fun: -10114.759310063291
     jac: array([-46873.,      0.,      0.,      0.])
 message: 'Optimization terminated successfully'
    nfev: 45
     nit: 11
    njev: 7
  status: 0
 success: True
       x: array([0.21579074, 0.2       , 0.11      , 0.049     ])

In [934]:
ineq1(sol.x)

9.999987913990644e-07

In [935]:
math.log2(8)

3.0